In [ ]:
import timeit
# records the time at this instant
# of the program
runtime_start = timeit.default_timer()

# 2 Running the SUMMA setups on HPC
In this notebook, we run SUMMA using pysumma, on the setups we created in the previous notebook.
This is the HPC version of the notebook and only runs the full problem, the most complex problem.
The last section of the notebook computes summary statistics of the output to be used in the next notebook. In this HPC notebook, you cannot see the summary statistics calculations as the code is on the HPC system.

The complexity choice is
 - 3)   `lhs_config_prob = 1`: 8 different configurations with the default and the exploration of the parameter space.


Less complex choices are available only in the local (non-HPC) version of the notebook, and not this notebook. These are, in order of increasing complexity: 
 - 1)   `default_prob = 1`: the "default" configuration with the "default" parameters. By "default" we mean whatever you chose in the summa setup files. 
 - 2a) `lhs_prob = 1`: the default configuration with exploration of the parameter space.
 - 2b) `config_prob = 1`: the default parameters with 8 different configurations (choices that have been seen to affect the model output in previous research) 
 

Eight iterations of each loop are run for each problem, to cover a truth run and the 7 forcings each held to a daily constant in turn.
 
You can make the problem run for fewer years to lower computational costs, changing `str(the_start)` and `str(the_end)`. You can test with as little as a day between `the_start` and `the_end`, but if you want to make the plots in this notebook and run the next notebook you will need >1 year (1 year is considered intialization period). 

## 2_1 Preliminary steps
### 2_1_1 Check the enviroment

<br>
Check that we loaded a correct environment. This should show pysumma version 3.0.3.

In [ ]:
conda list summa


### 2_1_2 Import libraries
Load the imports.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pysumma as ps
import xarray as xr
import pandas as pd
import gc
import os
import tempfile
import shutil
import json

### 2_1_3 Choose HPC machine

<div class="alert alert-block alert-danger">
<b> Choose the HPC  </b> Choose which HPC machine to use: either "expanse" or "keeling". We recommend using 'expanse' as it is a more poweful hpc leading to faster calculations.  </div>

In [ ]:
 hpc_machine = "expanse"

## 2_2 Set up paths to SUMMA configuration files for CAMELS basins
### 2_2_1 Set up paths to SUMMA configuration files
Set up the paths and regionalize the paths in the configuration files that SUMMA will use.

In [ ]:
top_folder = os.path.join(os.getcwd(), 'summa_camels')
settings_folder = os.path.join(top_folder, 'settings')
ps_working = os.path.join(top_folder, '.pysumma')
regress_folder = os.path.join(os.getcwd(), 'regress_data')

Make installTestCases_local.sh executable and navigate to it

In [ ]:
! cd {top_folder}; chmod +x installTestCases_local.sh; ./installTestCases_local.sh

In [ ]:
# get number of HRUs
attrib = xr.open_dataset(settings_folder+'/attributes.nc')
the_hru = np.array(attrib['hruId'])

<br>

##  2-3 Make problem complexity choices
 
The only complexity choice you can make is to run a different length time-period. It is pre-populated to run 6 years as in the paper. You also need to choose how long the initialization period is for the error calculations. We suggest 183 to 365 days, with at least 1 more year of simulation. So for example, if you run 18 months of simulation you should choose your initialization to be 183 days.


In [ ]:
default_prob = 0    #this should be 0 in this HPC notebook
lhs_prob = 0        #this should be 0 in this HPC notebook
config_prob = 0     #this should be 0 in this HPC notebook
lhs_config_prob = 1 #this should be 1 in this HPC notebook

<div class="alert alert-block alert-danger">
<b> Select simulation period and initialization days:  </b> Select simulation period and initialization days  </div>

In [ ]:
the_start = '1990-10-01 00:00' #pre-populated to '1990-10-01 00:00' as in the paper.
the_end =   '1996-09-30 23:00' #pre-populated to '1996-09-30 23:00' as in the paper.
initialization_days = 365 #pre-populated to 365 days as in the paper.

<br>

## 2_4 Setup HPC

This code sets up the HPC by setting the executable, writing the start and end files to the filemanager, and setting up workspaces on the HPC.

**TODO: Mention more details about the HPC that we use either here or at the beginning.**

In [ ]:
#define executable
executable =  '/usr/bin/summa.exe'

Write simulation start and end dates in the template_file_manager.1hr.txt file

In [ ]:
def replace_line_startwith(lines, flag, new_line_replacement):
    for i in range(len(lines)):
        if lines[i].startswith(flag):
            print(lines[i])
            lines[i] = new_line_replacement
    return lines
temple_filemanager = os.path.join(settings_folder, "template_file_manager.1hr.txt")
print(temple_filemanager)
with open(temple_filemanager, "r") as f:
    lines = f.readlines()
    new_lines1 = replace_line_startwith(lines, "simStartTime", "simStartTime         '{the_start}' !\n".format(the_start=the_start))
    new_lines2 = replace_line_startwith(new_lines1, "simEndTime", "simEndTime           '{the_end}' !\n".format(the_end=the_end))
f = open(os.path.join(settings_folder, "template_file_manager.1hr.txt"), "w")
#f.writelines(new_lines1)
f.writelines(new_lines2)
f.close()

   Special json encoder for numpy types

In [ ]:
class NumpyEncoder(json.JSONEncoder):
    """
    
    Credit:
    https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
    """
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.integer):
            return int(obj)
        return json.JSONEncoder.default(self, obj)

Handle summa_camels.zip

In [ ]:
!rm -rf summa_camels.zip
workspace_dir = os.path.join(os.getcwd(), 'workspace')
!mkdir -p {workspace_dir}
unzip_dir = tempfile.mkdtemp(dir=workspace_dir)
model_folder_name = "summa_camels"
model_folder = os.path.join(unzip_dir, model_folder_name)
shutil.make_archive(model_folder_name, 'zip', os.getcwd()+"/summa_camels")

<br>

## 2_5 Exploring the parameter  clibration space with a Latin Hypercube

Here we make parameter sets selected by using a Latin Hypercube to get 10 different parameter sets for every HRU, in order to explore the calibration space. The default parameter space is still included. This exploration will show us if the results of forcing importance could change after calibration. 

We change only the parameters that are usually calibrated. You can remove parameters if you were not planning to ever calibrate them away from their defaults (likewise you could add parameters).

The absolute minimums and maximums will break simulations and zero out variables, so we do not use those, we stay at the 5% level away from the extremes. Also, there are some constraints on the parameters that must be followed, they are:

* heightCanopyTop   > heightCanopyBottom
* critSoilTranspire > theta_res
* theta_sat         > critSoilTranspire
* fieldCapacity     > theta_res
* theta_sat         > fieldCapacity
* theta_sat         > theta_res
* critSoilTranspire > critSoilWilting
* critSoilWilting   > theta_res

In [ ]:
if lhs_prob==1 or lhs_config_prob==1: from pyDOE import lhs

In [ ]:
if lhs_prob==1 or lhs_config_prob==1:
    file_manager = settings_folder+'/file_manager_truth.txt'
    s = ps.Simulation(executable, file_manager)
    s.manager['simStartTime'] = str(the_start)
    s.manager['simEndTime'] = str(the_end)  
    #Before running the ensemble that changes parameters we must write the original simulation's parameters.
    s.manager.write()

In [ ]:
# print the default, min, and max as in /settings.v1/localParamInfo.txt and /settings.v1/basinParamInfo.txt
if lhs_prob==1 or lhs_config_prob==1:
    param_calib_hru = ['albedoRefresh', 'aquiferBaseflowExp', 'aquiferBaseflowRate', 'frozenPrecipMultip', 'heightCanopyBottom','heightCanopyTop', 'k_macropore', 
                   'k_soil', 'qSurfScale', 'summerLAI', 'tempCritRain', 'theta_sat', 'windReductionParam'] 
    param_calib_gru = ['routingGammaScale', 'routingGammaShape']

    for k in param_calib_hru:
        print(s.global_hru_params[k])
    for k in param_calib_gru:
        print(s.global_gru_params[k]) 

In [ ]:
if lhs_prob==1 or lhs_config_prob==1:
    bounds_hru = np.full((len(param_calib_hru),3),1.0)
    bounds_gru = np.full((len(param_calib_gru),3),1.0)
    for i,k in enumerate(param_calib_hru): bounds_hru[i,]= s.global_hru_params.get_value(k)[0:3]
    for i,k in enumerate(param_calib_gru): bounds_gru[i,]= s.global_gru_params.get_value(k)[0:3]

In [ ]:
# Define bounds and expand to size of LHS runs
if lhs_prob==1 or lhs_config_prob==1:
    numl = 10
    num_vars =  len(param_calib_hru) + len(param_calib_gru)
    names =  param_calib_hru + param_calib_gru
    bounds =  np.concatenate((bounds_hru, bounds_gru), axis=0)
    par_def = dict(zip(names, np.transpose(np.tile(bounds[:,0],(len(the_hru),1))) ))
    par_min = dict(zip(names, np.transpose(np.tile(bounds[:,1],(numl*len(the_hru),1))) ))
    par_max = dict(zip(names, np.transpose(np.tile(bounds[:,2],(numl*len(the_hru),1))) ))

<br>
We remove geographically distributed parameters from the default set, and then make the LHS parameter set plus deault from the above bounds. Set 0 will be the default parameter set.

In [ ]:
# remove geographically distributed parameters from default set
if lhs_prob==1 or lhs_config_prob==1:
    distributed_val = par_def.pop('heightCanopyBottom')
    distributed_val = par_def.pop('heightCanopyTop')
    distributed_val = par_def.pop('k_soil')
    distributed_val = par_def.pop('theta_sat')

In [ ]:
# Make sure to obey parameter constraints
if lhs_prob==1 or lhs_config_prob==1:
    param = xr.open_dataset(settings_folder+'/parameters.nc')

    for i,h in enumerate(the_hru):
        lb_theta_sat = max(param[['critSoilTranspire','fieldCapacity','theta_res']].isel(hru=i).values()).values
        for j in range(0,numl): #say first numl belong to hru 0, second numl to hru 1, and so on
            if (par_min['theta_sat'][j + i*numl]<lb_theta_sat): par_min['theta_sat'][j + i*numl]=lb_theta_sat

    par_min['heightCanopyTop'] = par_max['heightCanopyBottom']

In [ ]:
# Add a 5% buffer
if lhs_prob==1 or lhs_config_prob==1:
    buff = {key: (par_max.get(key) - par_min.get(key))*0.05 for key in set(par_max) }
    par_min = {key: par_min.get(key) + buff.get(key)*0.05 for key in set(buff) }
    par_max = {key: par_max.get(key) - buff.get(key)*0.05 for key in set(buff) }

In [ ]:
#Generate samples with Latin Hypercube Sampling, set seed by HRU ID so it is the same every time run
if lhs_prob==1 or lhs_config_prob==1:
    lhd = np.empty(shape=(num_vars,numl*len(the_hru)))
    for i, h in enumerate(the_hru):
        np.random.seed(h) #if the hru ID is not a number this will not work
        lhd[:,range(i*numl,(i+1)*numl)] = lhs(numl, samples=num_vars)
    lhd = dict(zip(names,lhd))
    samples = {key: par_min.get(key) + lhd.get(key)*(par_max.get(key) - par_min.get(key)) for key in set(par_max) }

In [ ]:
# make parameter sets
if lhs_prob==1 or lhs_config_prob==1:
    latin = {}
    latin[str(0)] = {'trial_parameters': {key: par_def.get(key) for key in set(par_def) }}
    for j in range(0,numl):
            latin[str(j+1)] = {'trial_parameters': {key: samples.get(key)[np.arange(j, len(the_hru)*numl, numl)] for key in set(samples) }}

<br>

## 2_6 Manipulating the configuration of the pysumma objects

We need to run the parameter space with other model configurations, to see if the results seen on the default configuration hold true across the parameter space. The new configurations follow the exploration of [this paper](https://doi.org/10.1002/2015WR017200).

Clark, M.P., Nijssen, B., Lundquist, J.D., Kavetski, D., Rupp, D.E., Woods, R.A., Freer, J.E., Gutmann, E.D., Wood, A.W., Gochis, D.J. and Rasmussen, R.M., 2015. A unified approach for process‐based hydrologic modeling: 2. Model implementation and case studies. Water Resources Research, 51(4), pp.2515-2542.

Of the model configurations discussed in this paper, the decisions that made the most difference are:

 - `groundwatr` choice of groundwater parameterization as:
   - `qTopmodl` the topmodel parameterization (note must set hc_profile = pow_prof and bcLowrSoiH = zeroFlux
   - `bigBuckt` a big bucket (lumped aquifer model) in between the other two choices for complexity
   - `noXplict` no explicit groundwater parameterization
 - `stomResist` choice of function for stomatal resistance as:
   - `BallBerry` Ball-Berry (1987) parameterization of physiological factors controlling transpiration
   - `Jarvis` Jarvis (1976) parameterization of physiological factors controlling transpiration
   - `simpleResistance` parameterized solely as a function of soil moisture limitations
 - `snowIncept` choice of parameterization for snow interception as:
   - `stickySnow` maximum interception capacity is an increasing function of temperature
   - `lightSnow` maximum interception capacity is an inverse function of new snow density
 - `windPrfile` choice of wind profile as:
   - `exponential` an exponential wind profile extends to the surface
   - `logBelowCanopy` a logarithmic profile below the vegetation canopy

Choices `bigBuckt`, `BallBerry`, `lightSnow`, and `logBelowCanopy` are the defaults that we have run already (see the decisions printed out in the previous cell). The paper showed choice of `groundwatr` affecting the timing of runoff and the magnitude of evapotranspiration, `stomResist` affecting timing and magnitude of evapotranspiration, `snowIncept` affecting the magnitude canopy interception of snow, and `windPrfile` affecting the timing and magnitude of SWE, and latent and sensible heat. We will not explore the `groundwatr` configurations here as the differences show up only in most simulated variables post-calibration. This study does not examine models calibrated for every set up of forcings configurations. Note, if you want to look at `qTopmodl`, you must set  `bcLowrSoiH` to `zeroFlux` (we will leave it at `drainage`) and `hc_profile` to `pow_prof` (we will leave it at `constant`). You can add other configuration choices here; this notebook and the next notebook will work properly (but it will make the computations take longer).

We make use of pysumma `Simulation` objects and the `Ensemble` class, to set up suites of different model decisions (and add to this the different sets of parameters in the next section).

In [ ]:
if config_prob or lhs_config_prob==1:
    file_manager = settings_folder+'/file_manager_truth.txt'
    s = ps.Simulation(executable, file_manager)
    s.manager['simStartTime'] = str(the_start)
    s.manager['simEndTime'] = str(the_end)  
    #Before running the ensemble that changes configuration we must write the original simulation's configuration.
    s.manager.write()    
    print(s.decisions)

In [ ]:
if config_prob or lhs_config_prob==1:
    #alld = {'groundwatr':np.array(['qTopmodl','bigBuckt']),
    alld = {'stomResist':np.sort(np.array(['BallBerry','Jarvis'])),
            'snowIncept':np.sort(np.array(['stickySnow', 'lightSnow'])),
            'windPrfile':np.sort(np.array(['exponential','logBelowCanopy']))}
    config = ps.ensemble.decision_product(alld)

<br>
The ensemble uses `++` as a delimiter to create unique identifiers for each simulation in the ensemble. The default configuration will be run again. We do this so that each finished SUMMA *.nc output file is complete.

<br>

## 2_7 Run the full problem

Now we can use the following code to make the full problem, exploring the parameter space and the configurations together. This is the problem that is run on the HPC.
The next notebook in the series runs the most complete figures using this full problem.


### 2_7_1 Combine the decision sets

In [ ]:
# make ensembles with parameter space (numl parameter sets plus 1 for default), should make 88
if lhs_config_prob==1:
    config_latin = {}
    for key_config in config.keys():
        c = config[key_config]
        for key_latin in latin.keys():
            l = latin[key_latin]
            config_latin[key_config+key_latin] = {**c,**l}
    print(len(config_latin))

In [ ]:
config_latin1 = json.dumps(config_latin, cls=NumpyEncoder)
config_latin = json.loads(config_latin1)
list(config_latin.items())[:2]

<br>

### 2_7_2 HPC setup

In [ ]:
constant_vars= ["truth",
                'constant_airpres','constant_airtemp','constant_LWRadAtm',
                'constant_pptrate','constant_spechum','constant_SWRadAtm',
                'constant_windspd', ]
config = {}
idx = 0 
for i, v in enumerate(constant_vars):
    if v == "truth":
        key = v
    else:
        key = 'run'+str(idx)
        idx+=1
    value = {'file_manager': '<PWD>/settings/file_manager_' + v +'.txt'}
    config[key] = value
config

In [ ]:
config_3 = {}
for k, v in config.items():
    for k2, v2 in config_latin.items():
        v_copy = v.copy()
        v_copy.update(v2)
        config_3["{}_{}".format(k, k2)] = v_copy
len(config_3)

In [ ]:
config_3 = json.dumps(config_3, cls=NumpyEncoder)
config_3 = json.loads(config_3)
list(config_3.items())[:1]

In [ ]:
list(config_3.items())[-1]

**TODO: Add explanation for the next cell**

In [ ]:
if lhs_config_prob==1:
    import tempfile
    import shutil, os
    workspace_dir = os.path.join(os.getcwd(), 'workspace')
    !mkdir -p {workspace_dir}
    unzip_dir = tempfile.mkdtemp(dir=workspace_dir)
    model_folder_name = "summa_camels"
    model_folder = os.path.join(unzip_dir, model_folder_name)
    !unzip -o {model_folder_name}.zip -d {model_folder}
    !rm -rf {model_folder}/output
    !mkdir {model_folder}/output
    !mkdir {model_folder}/output/constant
    !mkdir {model_folder}/output/merged_day
    !mkdir {model_folder}/output/truth
    !mkdir {model_folder}/output/regress_data
    with open(os.path.join(model_folder, "output/regress_data/regress_param.json"), 'w') as f:
        json.dump({"initialization_days": initialization_days}, f)

**TODO: Add explanation for the next cell**

In [ ]:
if lhs_config_prob==1:
    import json
    with open(os.path.join(model_folder, 'summa_options.json'), 'w') as outfile:
        json.dump(config_3, outfile)

    # check ensemble parameters    
    print("Number of ensemble runs: {}".format(len(config_3)))
    print(json.dumps(config_3, indent=4, sort_keys=True)[:800])
    print("...")

<br>

### 2_7_3 Submit model to HPC using CyberGIS-Compute Service

<div class="alert alert-block alert-info">
<b> Start HPC use  </b> (config_latin) config_latin SUMMA runs with each forcing </div>

In [ ]:
if lhs_config_prob==1:
    from job_supervisor_client import *
    communitySummaSession = Session('summa', isJupyter=True)
    communitySummaJob = communitySummaSession.job() # create new job
    communitySummaJob.upload(model_folder)

<div class="alert alert-block alert-danger">
<b> Set the walltime  </b> If you are running for multiple basins, conisder increasing `"walltime" : 5` (which is the cap run time on the hpc in hours), to higher numbers so that the running time on the hpc does not extend this value. Otherwise, the job will fail. </div>

In [ ]:
if lhs_config_prob==1:
    communitySummaJob.submit(payload={
        "node": 256,
        "machine":  hpc_machine,
        "walltime" : 5,
        "file_manager_rel_path": "settings/file_manager_constant_airpres.txt"
    })

In [ ]:
%%time
if lhs_config_prob==1:
    communitySummaJob.events(liveOutput=True)

* When outputs are very large, sometimes missing data happens. In that case, try to execute the above cell again.


Next, create output folder, and then download the file containing KGE error on outputs.

In [ ]:
%%time
if lhs_config_prob==1:
    job_dir = os.path.join(model_folder, "{}".format(communitySummaJob.id))
    !mkdir -p {job_dir}/output
    communitySummaJob.download(job_dir)

In [ ]:
!cd {job_dir} && unzip -o *.zip -d output

In [ ]:
error_path = os.path.join(job_dir, "output/regress_data/error_data_configs_latin.nc")
print(error_path)
xr.open_dataset(error_path)

In [ ]:
! mkdir -p ./regress_data
! cp {error_path} ./regress_data

<div class="alert alert-block alert-info">
<b> Finish HPC use  </b> (config_latin) config_latin SUMMA runs with each forcing </div>

<br>

## 2_8 Compute KGE error on outputs
We calculate KGE statistics on the data. KGE means perfect agreement if it is 1, and <0 means the mean is a better guess. We use a modified KGE that avoids the amplified simulated mean divided by truth mean values when is the truth mean is small, and avoids the dependence of the KGE metric on the units of measurement. Then, we scale the KGE so that the range is 1 to -1.
If the values are identical we use KGE of 1.
We also keep summaries of the raw data (summed over time). 
This can take some time depending on how big of a problem you ran. It takes about 1/100th of the time it took to run the whole problem. 


All KGE codes were moved over to the CyberGIS-Compute Service and will execute on HPC. The codes can be found here.

[https://github.com/cybergis/Jupyter-xsede/blob/master/cybergis/summa.py#L268](https://github.com/cybergis/Jupyter-xsede/blob/master/cybergis/summa.py#L268)

In [ ]:
# Print the time spent running the entire notebook.
# records the time at this instant 
# of the program
runtime_end = timeit.default_timer()

# printing the execution time by subtracting
# the time before the function from
# the time after the function
print(int(runtime_end-runtime_start), ' seconds')